In [11]:
print("\tVERSION INFORMATION")
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_io as tfio; print(f"\t\t– TENSORFLOW I/O VERSION: {tfio.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;

	VERSION INFORMATION
		– TENSORFLOW VERSION: 2.9.0
		– TENSORFLOW I/O VERSION: 0.26.0


In [12]:
import os
from glob import glob
import pandas as pd

In [13]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
DATA_DIR = "D:Dataset/dataset/hubmap-organ-segmentation/"
for file in tf.io.gfile.glob(os.path.join(DATA_DIR, "*")): print(f"\t--> {file}")

	--> D:Dataset\dataset\hubmap-organ-segmentation\masked_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\sample_submission.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\test_images
	--> D:Dataset\dataset\hubmap-organ-segmentation\train.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_annotations
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_fix.csv
	--> D:Dataset\dataset\hubmap-organ-segmentation\train_images


In [15]:
train_df = pd.read_csv(DATA_DIR+"train_fix.csv")

In [16]:
train_df

,id,organ,img_height,pixel_size
0,10044,prostate,3000,0.4
1,10274,prostate,3000,0.4
2,10392,spleen,3000,0.4
3,10488,lung,3000,0.4
4,10610,spleen,3000,0.4
...,...,...,...,...
346,9517,kidney,3000,0.4
347,9769,kidney,3070,0.4
348,9777,largeintestine,3000,0.4
349,9791,kidney,3000,0.4


In [19]:
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True) # XLA은 텐서플로우 환경안에서 도메인 별로 특화된 선형대수 계산 컴파일러이며, 연산 속도와 메모리 사용량을 향상시킨다.

In [20]:
def rle2mask(rle, width, target_size=None):
    if target_size == None:
        target_size = width

    rle = np.array(list(map(int, rle.split())))
    label = np.zeros((width*width))
    
    for start, end in zip(rle[::2], rle[1::2]):
        label[start:start+end] = 1
        
    #Convert label to image
    label = Image.fromarray(label.reshape(width, width))
    #Resize label
    label = label.resize((target_size, target_size))
    label = np.array(label).astype(float)
    #rescale label
    label = np.round((label - label.min())/(label.max() - label.min()))
    
    return label.T